In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
#import preprocessor as p
import re
import warnings
warnings.filterwarnings("ignore")
from bs4 import BeautifulSoup
import string
import html
import requests
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('punkt')
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import os
import pickle
from sklearn.metrics import classification_report, accuracy_score
from google.colab import files
import io

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
!pip install preprocessor
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for preprocessor: filename=preprocessor-1.1.3-py3-none-any.whl size=4473 sha256=3f71d7eb23fcab5ad112d26c4a86552b0b6bb78412e7bec61a8868b20278f9ea
  Stored in directory: /root/.cache/pip/wheels/18/41/3e/30e0c08b0759166f31aee174d58d288ad5c277bd102c8ad869
Successfully built preprocessor
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 52.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
uploaded = files.upload()
df_russia_may = pd.read_csv(io.BytesIO(uploaded['tweet_russia_may.csv']))

Saving tweet_russia_may.csv to tweet_russia_may.csv


In [4]:
uploaded = files.upload()
df_russia_jun = pd.read_csv(io.BytesIO(uploaded['tweet_russia_jun.csv']))

Saving tweet_russia_jun.csv to tweet_russia_jun.csv


In [5]:
uploaded = files.upload()
df_russia_jul = pd.read_csv(io.BytesIO(uploaded['tweet_russia_jul.csv']))

Saving tweet_russia_jul.csv to tweet_russia_jul.csv


In [6]:
col = ["Unnamed:0", "ID", "Date", "Tweet"]
df_russia = pd.DataFrame(columns=col)

df_russia = pd.concat([df_russia, df_russia_may])
df_russia = pd.concat([df_russia, df_russia_jun])
df_russia = pd.concat([df_russia, df_russia_jul])

In [7]:
def clean_text(_sentence):

    #removes emojis, pictographs, map symbols and flags
    emoji_pattern = re.compile(pattern = "["
          u"\U0001F600-\U0001F64F" 
          u"\U0001F300-\U0001F5FF" 
          u"\U0001F680-\U0001F6FF" 
          u"\U0001F1E0-\U0001F1FF" 
                            "]+", flags = re.UNICODE)
    _sentence = emoji_pattern.sub(r'',_sentence)

    #emoves tags and attributes
    _sentence = re.sub('(<[^>]*>)','',_sentence)

    #converts character code to string
    _sentence = html.unescape(_sentence)

    #removes urls
    _sentence = re.sub("(\w+:\/\/\S+)|(http[^\s]+)|(www.[^\s]+)",'',_sentence, flags=re.MULTILINE)
    
    #removes RT
    _sentence = re.sub("RT",'',_sentence)
    
    #converts the sentence to a lower case
    _sentence = _sentence.lower()

    #removes punctuation
    _sentence = re.sub(r'\@\w+|\#\w+','', _sentence)
    _sentence = _sentence.translate(str.maketrans('', '', (string.punctuation)))
    
    #checks for stop words
    stop_words = set(stopwords.words('english'))
    _sentence = [word for word in str(_sentence).split() if word not in stop_words]

    #joins the word and returns a sentence
    return " ".join(_sentence)

#function to preprocess tweets
def preprocess_tweets(tweets_df,column):

    #apply clean text in a vectorized format
    vfunc_clean_text = np.vectorize(clean_text)
    tweets_df[column] = vfunc_clean_text(tweets_df[column]) #create a new column that consists of preprocessed tweets
    
    return tweets_df

In [8]:
data_df_russia = preprocess_tweets(df_russia,'Tweet')

In [9]:
def Vader_process(data_df):
    sid = SentimentIntensityAnalyzer()
    res = [*data_df['Tweet'].apply(sid.polarity_scores)]
    #print(res[:3])

    sentiment_df = pd.DataFrame.from_records(res)
    data_df = pd.concat([data_df.reset_index(drop=True), sentiment_df.reset_index(drop=True)], axis=1, join="inner")
    #print(data_df.head())
    
    conditions = [
    (data_df['compound'] <= -0.5),
    (data_df['compound'] > -0.5) & (data_df['compound'] < 0.5),
    (data_df['compound'] >= 0.5)
    ]

    values = [0, -1, 1]
    data_df['label'] = np.select(conditions, values)

    #print(data_df.head())
    
    X_df = data_df[data_df.label != -1]
    X_df = X_df[['Tweet','label']]
    #print(X_df.head())
    
    return X_df

In [10]:
X_russia = Vader_process(data_df_russia)

In [11]:
X_russia

,Tweet,label
1,mean fbi cia nsa flunkies lied fisa warrant ap...,0
2,im sure democrat room disparage anyone concern...,0
4,ive got friends ukraine better believe ive tha...,1
8,putin doesnt ask anything ppl want come fight ...,0
10,sheer hypocrisy left “we oppose aggression occ...,0
...,...,...
505985,russia consistently argued taking action behal...,0
505986,budapest memorandum russia promised respect in...,1
505990,jun 1941 still alles arming getting ready move...,0
505997,russias intent destroy ukranians economy huge ...,0


**BERTopic Modelling**

In [12]:
import nltk
from nltk.corpus import stopwords
from umap import UMAP
from bertopic import BERTopic

In [13]:
umap_model = UMAP(random_state=3231)
bert_topic_model= BERTopic(embedding_model = "all-MiniLM-L6-v2", umap_model=umap_model)

bert_negative = X_russia[X_russia['label']==0] #change this to 1 if positive. keep 0 if negative.
bert_negative = bert_negative.Tweet.tolist()
topics, probs = bert_topic_model.fit_transform(bert_negative)

In [14]:
pd.set_option('display.max_rows', None)


In [15]:
topic_list=bert_topic_model.get_topic_info()
topic_list

,Topic,Count,Name
0,-1,64760,-1_russian_oil_people_ukraine
1,0,4206,0_hoax_collusion_hillary_election
2,1,4098,1_nato_expansion_natos_join
3,2,3098,2_grain_food_wheat_ports
4,3,1388,3_nukes_nuclear_nuke_use
5,4,1351,4_sanctions_imposed_hurt_hurting
6,5,1306,5_china_chinese_chinas_xi
7,6,953,6_finland_sweden_joining_finns
8,7,920,7_zelensky_zelenskyy_zelenskys_volodymyr
9,8,781,8_biden_joe_bidens_admin


In [16]:
bert_topic_model.visualize_barchart()

In [17]:
#Positive
umap_model = UMAP(random_state=3231)
bert_topic_model= BERTopic(embedding_model = "all-MiniLM-L6-v2", umap_model=umap_model)

bert_positive = X_russia[X_russia['label']==1] #change this to 1 if positive. keep 0 if negative.
bert_positive = bert_positive.Tweet.tolist()
topics, probs = bert_topic_model.fit_transform(bert_positive)

pd.set_option('display.max_rows', None)
topic_list=bert_topic_model.get_topic_info()
topic_list

bert_topic_model.visualize_barchart()